In [ ]:
'''
这里的mark改了一下。。你对自己做的是35个单词一组。

分散 103
集中 102
一次 101
206-240

后面打的mark
 10 11

  自信程度

20 - 25pp

'''

'\n这里的mark改了一下。。你对自己做的是35个单词一组。\n\n分散 103\n集中 102\n一次 101\n206-240\n\n后面打的mark\n 10 11\n\n  自信程度\n\n20 - 25\n\n'

In [ ]:
import os.path as op
import mne
from mne.preprocessing import ICA
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np

from autoreject import AutoReject

In [ ]:
sub_ids = [
            'prex006','prex009','prex010','prex011','prex012',
                        'prex015','prex016','prex017','prex018','prex019','prex022','prex023',
                        'prex026','prex027','prex028'
                        ,'prex033','prex034','prex037','prex038','prex039','prex040',
                        'prex041','prex042','prex043','prex044','prex045',
                        'prex046','prex047','prex048','prex049','prex050'
          ]

sub_ids = [ 'prex006' ]
evoked_original = list()
evoked_clean = list()
for sub_id,i in zip(sub_ids,range(len(sub_ids))):
    #读入之前处理好的reconst_raw
    save_path_ica = 'E:\Python LYW\RSA\spaced_learning\pre10\preprocessing\data\\4raw_ica\\'
    reconst_raw=mne.io.read_raw_fif(save_path_ica + sub_id + '.fif')
    #截取block0的时间段，每个被试不一样。。你要换个mark才行。
    #或者有什么办法读取一个segment。
    # reconst_raw = reconst_raw.crop(tmax = 515.480)
    # reconst_raw = reconst_raw.crop(tmax = 1159.360)
    #这个是被试测试一的。
    #epoch
    target_events = {
                     'Stimulus/S101':101, 'Stimulus/S102':102, 'Stimulus/S103':103,
                     'Stimulus/S 10':10,'Stimulus/S 11':11,'Stimulus/S 20':20 ,
                     'Stimulus/S 21':21,'Stimulus/S 22':22,'Stimulus/S 23':23,
                     'Stimulus/S 24':24,'Stimulus/S 25':25}

    #先找events，epoch和average都是一步的事情。
    custom_mapping = target_events
    (events_from_annot,
     event_dict) = mne.events_from_annotations(reconst_raw, event_id=custom_mapping)
    #还不如直接对 events_from_annot 进行切片
    #先遍历，把合适的单词筛序出来
    #对应的vocabulary已经列出，然后怎么提取呢？

    events=[]

    voc_mark = 101
        # print(voc_mark)
        #遍历所有，再做一次筛选mark
    for j_event in range(len(events_from_annot)-1):
            # if ((events_from_annot[j_event,2] == voc_mark and events_from_annot[j_event+1,2] == 16)or(events_from_annot[j_event,2]==voc_mark and events_from_annot[j_event+1,2]==11)):
            #全部叠加，不考虑是否能够recognize。
        if events_from_annot[j_event,2] == voc_mark and events_from_annot[j_event+1,2] == 11 and events_from_annot[j_event+2,2]>=21:
            events.append(events_from_annot[j_event,:])
            print(events_from_annot[j_event,:])
    count_remembered_events = len(events)
    #取全部event对应的epochs
    # _, eog_scores= icas[i].find_bads_eog(raws[i])
    # ica.plot_scores(eog_scores)
    #顺带对比ica前后的效果。
    epochs = mne.Epochs(reconst_raw, events, baseline =(-0.2,0),tmin=-0.2, tmax=0.8, event_id=None,
                        preload=True, event_repeated='merge')

    save_path_epoch = 'E:\Python LYW\RSA\spaced_learning\pre10\preprocessing\data\\5epoch\\'
    epochs.save(save_path_epoch + sub_id + '-epo.fif', overwrite=True)


    #用autoreject处理一下
    n_interpolates = np.array([1, 4, 32])
    consensus_percs = np.linspace(0, 0.8, 11)
    picks = mne.pick_types(epochs.info, meg=False, eeg=True, stim=False, eog=False,
                           include=[], exclude=[])
    ar = AutoReject(n_interpolates,consensus_percs,picks=picks,
                    thresh_method='random_search',random_state=23)
    ar.fit(epochs)
    epochs_clean = ar.transform(epochs)


    #先average看下结果，对比
    #如果结果不错就可以分别保存下来。
    evoked_original.append(epochs.average())
    evoked_clean.append(epochs_clean.average())

    save_path_epoch_clean = 'E:\Python LYW\RSA\spaced_learning\pre10\preprocessing\data\\6epoch_clean\\'
    epochs_clean.save(save_path_epoch_clean + sub_id + '-epo.fif', overwrite=True)
    save_path_evoked = 'E:\Python LYW\RSA\spaced_learning\pre10\preprocessing\data\\7evoked\\'
    #为什么这里加i?因为是list，epochs不是list
    evoked_clean[i].save(save_path_evoked + sub_id + '-ave.fif')

    # fig_original = evoked_original[i].plot(window_title = sub_id)
    # fig_original.savefig(save_path_evoked+'_original_' + sub_id)
    # fig_clean = evoked_clean[i].plot(window_title = sub_id)
    # fig_clean.savefig(save_path_evoked+'_clean_'+ sub_id)


In [ ]:
fig_original = evoked_original[i].plot(window_title = sub_id)
fig_original.savefig(save_path_evoked+'_original_' + sub_id)
fig_clean = evoked_clean[i].plot(window_title = sub_id)
fig_clean.savefig(save_path_evoked+'_clean_'+ sub_id)

In [ ]:
evoked_clean.plot()
